In [1]:
import pandas as pd
import numpy as np

In [2]:
cagedf = pd.read_csv('https://raw.githubusercontent.com/torarm/portfolio-project-1/main/cagemovies%20-%20cagemovies.csv')
coppoladf = pd.read_csv('https://raw.githubusercontent.com/torarm/portfolio-project-1/main/coppolamovies%20-%20coppolamovies.csv')

In [3]:
#Removing films that Cage remade & drop extra column

cagedf = cagedf.drop([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], axis=0)
cagedf = cagedf.drop(['Unnamed: 0'], axis=1)
cagedf = cagedf.reset_index(drop=True)

coppoladf = coppoladf.drop(['Unnamed: 0'], axis=1)

In [4]:
#Removing the 3 films FFC directed and Cage acted in

bothmovies = ['Rumble Fish', 'The Cotton Club', 'Peggy Sue Got Married']
cagedf = cagedf[~cagedf['original_title'].isin(bothmovies)]
coppoladf = coppoladf[~coppoladf['original_title'].isin(bothmovies)]

In [5]:
#Total votes per rating and variable for total votes
sum_cage = cagedf[['votes_10', 'votes_9',
       'votes_8', 'votes_7', 'votes_6', 'votes_5', 'votes_4', 'votes_3',
       'votes_2', 'votes_1']].sum(axis=0)
total_cage = cagedf['total_votes'].sum()

sum_cop = coppoladf[['votes_10', 'votes_9',
       'votes_8', 'votes_7', 'votes_6', 'votes_5', 'votes_4', 'votes_3',
       'votes_2', 'votes_1']].sum(axis=0)
total_cop = coppoladf['total_votes'].sum()

In [6]:
#Functions for distribution of ratings
def div_total(int):
  return int / total_cage
sum_cage_total = sum_cage.apply(div_total)

def div_total2(int):
  return int / total_cop
sum_cop_total = sum_cop.apply(div_total2)

In [7]:
#Series to DF with rating as categorical variable
cagevotesdf = pd.DataFrame({'votes': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 'Cage': list(sum_cage_total)
})
copvotesdf = pd.DataFrame({'votes': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 'Coppola': list(sum_cop_total)})

In [8]:
#Sort by rating value
cagevotesdf = cagevotesdf.sort_values(['Cage']).reset_index(drop=True)
copvotesdf = copvotesdf.sort_values(['Coppola']).reset_index(drop=True)

In [9]:
#Combine dataframes & melt
combined_votes = pd.merge(cagevotesdf, copvotesdf, on='votes')
votemelt = pd.melt(combined_votes, id_vars=['votes'], value_vars=['Cage', 'Coppola'], var_name='name')

In [10]:
#Rating distribution
import plotly.express as px

fig = px.bar(votemelt, x="votes", y="value", color="name", title="IMDb Rating Distribution")
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

In [11]:
#Average cumulative votes per movie
import plotly.graph_objects as go

x = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
fig = go.Figure(data=[
    go.Bar(name='Nick Cage', x=x, y=(sum_cage / 100)),
    go.Bar(name='Copppola', x=x, y=(sum_cop / 32))
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title_text='Cumulative Rating per Movie')
fig.show()

In [73]:
from plotly.subplots import make_subplots

labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Cage Movies', 'Coppola Movies'])
fig.add_trace(go.Pie(labels=labels, values=cagevotesdf['Cage'], scalegroup='one', name='Cage'), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=copvotesdf['Coppola'], scalegroup='one', name='Coppola'), 1, 2)

fig.update_layout(title_text='IMDb Rating Distribution of Total Vote')
fig.update_traces(hoverinfo='value', textinfo='label+percent',
                  marker=dict(colors=px.colors.sequential.RdBu))

fig.show()

In [12]:
#For hypothesis testing
from scipy import stats

copmean = coppoladf['mean_vote'].mean()
copstd = coppoladf['mean_vote'].std()
ncop = len(coppoladf)

cagemean = cagedf['mean_vote'].mean()
cagestd = cagedf['mean_vote'].std()
ncage = len(cagedf)

In [13]:
#Two-sample t-test
np.random.seed(42)
rvs1 = stats.norm.rvs(loc=copmean, scale=copstd,size=ncop)
rvs2 = stats.norm.rvs(loc=cagemean, scale=cagestd, size=ncage)

stats.ttest_ind(rvs2, rvs1)

Ttest_indResult(statistic=-1.783231323618395, pvalue=0.07699511965595147)

t-stat= -1.783

pvalue=0.077

In [48]:
#Subset for box office
copboxoffice = coppoladf[['year', 'usa_gross_income', 'worlwide_gross_income']].dropna()
cageboxoffice = cagedf[['year', 'usa_gross_income', 'worlwide_gross_income']].dropna()

#Clean
cageboxoffice[cageboxoffice.columns[0:]] = cageboxoffice[cageboxoffice.columns[0:]].replace('[\$,]', '', regex=True).astype(float)
copboxoffice[copboxoffice.columns[0:]] = copboxoffice[copboxoffice.columns[0:]].replace('[\$,]', '', regex=True).astype(float)

In [49]:
#Convert year to date-time object
cageboxoffice['year'] = cageboxoffice['year'].astype('int')
cageboxoffice['year'] = pd.to_datetime(cageboxoffice['year'].astype(str), format='%Y')

copboxoffice['year'] = copboxoffice['year'].astype('int')
copboxoffice['year'] = pd.to_datetime(copboxoffice['year'].astype(str), format='%Y')

In [50]:
#Remove months
cageboxoffice['year'] = cageboxoffice['year'].dt.year
copboxoffice['year'] = copboxoffice['year'].dt.year

In [52]:
#Sort by year
cageboxoffice = cageboxoffice.sort_values(['year']).reset_index(drop=True)
copboxoffice = copboxoffice.sort_values(['year']).reset_index(drop=True)

In [54]:
#Total income column

copboxoffice['total_income'] = copboxoffice['usa_gross_income'] + copboxoffice['worlwide_gross_income']
cageboxoffice['total_income'] = cageboxoffice['usa_gross_income'] + cageboxoffice['worlwide_gross_income']

In [108]:
#Hypothesis test vars
ncopbox = len(copboxoffice)
meancopbox = copboxoffice['total_income'].mean()
copboxstd = copboxoffice['total_income'].std()

ncagebox = len(cageboxoffice)
meancagebox = cageboxoffice['total_income'].mean()
cageboxstd = cageboxoffice['total_income'].std()

In [109]:
# Two sample t-test

np.random.seed(42)
rvs_ca = stats.norm.rvs(loc=meancagebox, scale=cageboxstd, size=ncagebox)
rvs_co = stats.norm.rvs(loc=meancopbox, scale=copboxstd, size=ncopbox)

stats.ttest_ind(rvs_ca, rvs_co)

Ttest_indResult(statistic=0.4290331299272621, pvalue=0.6689845556885146)

In [78]:
fig = go.Figure(data=[
    go.Bar(name='Cage', x=cageboxoffice['year'], y=cageboxoffice['total_income']),
    go.Bar(name='Coppola', x=copboxoffice['year'], y=copboxoffice['total_income'])
])

fig.update_layout(barmode='group')
fig.update_layout(title_text='Total Box Office Income by Year')
fig.show()

In [107]:
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=cageboxoffice['total_income'],
    histnorm='percent',
    name='Cage', 
    xbins=dict( 
        start=0,
        end=500000000,
        size=50000000
    ),
    marker_color='red',
    opacity=0.85
))
fig.add_trace(go.Histogram(
    x=copboxoffice['total_income'],
    histnorm='percent',
    name='Coppola',
    xbins=dict(
        start=0,
        end=500000000,
        size=50000000
    ),
    marker_color='goldenrod',
    opacity=0.85
))

fig.update_layout(
    title_text='Sampled Distribution of Total Box Office Returns', 
    xaxis_title_text='Total Income in Dollars', 
    yaxis_title_text='Percent', 
    bargap=0.1, 
    bargroupgap=0.05 
)

fig.show()